In [4]:
import sys
import os
from accelerate import notebook_launcher
from datasets import load_dataset
import logging

# Configure logging:
# (i) lear any pre-existing logging handlers (especially useful in notebook reruns)
for handler in logging.root.handlers[:]:
    logging.root.removeHandler(handler)
# (ii) now safely configure logging
logging.getLogger("codecarbon").setLevel(logging.ERROR)
logging.basicConfig(level=logging.INFO)

# Adjust paths -> import classes
project_root = os.getcwd()  
if project_root not in sys.path:
    sys.path.append(project_root)
    
from configs.experiment_config import ExperimentConfig
from experiment_orchestration.experiment_runner import ExperimentRunner

#import torch.multiprocessing as mp
#mp.set_start_method('spawn', force=True)

In [5]:
experiment_config = ExperimentConfig(
    model_name="TinyLlama/TinyLlama-1.1B-Chat-v1.0",
    is_encoder_decoder=False,
    task_type="text_generation",
    inference_type="pure_generative",  
    max_input_tokens=500, #2048 is Llama's limit
    max_output_tokens=200,
    num_input_prompts=10,
    save_outputs=True,
    decode_token_to_text=True,
    gpu_list=[0,1,2,3],
    num_processes=4,
    batching_options={
        "batch_size___fixed_batching":16,
        "adaptive_batching": False,
        "adaptive_max_tokens": 3000,
        "max_batch_size___adaptive_batching": 100
    },
    sharding_config={
        "fsdp_config": {
            "use_orig_params": True,
            "cpu_offload": True
        },
        "sharding_strategy": "NO_SHARD" #FULL_SHARD doesn't work
    },
    query_rate=1,
    decoder_temperature=1.0, # if 0: deterministic, if >0 do_sample is activated. NB: needs to be a float.. Higher Temperature → Longer Outputs (sometimes)If stopping criterion is based on tokens (e.g., EOS token), high temperature might delay reaching it.

    fp_precision="float32",
    quantization_config={
        "quantization": False,
        "cached_flops_for_quantised_models": 5172720640000, # THIS IS NEEDED UNTIL I CAN WORK OUT HOW TO COMPUTE FLOPS OF QUANITSED MODELS
        "load_in_8bit": False,
        "load_in_4bit": True,
        #"llm_int8_threshold"=6.0,
        "llm_int8_enable_fp32_cpu_offload": False,
        "llm_int8_has_fp16_weight": False,
    },
    backend="pytorch"
)


In [ ]:

# Load prompts (here, using the 'arxiv' split from the lighteval/pile_helm dataset)
ds = load_dataset("lighteval/pile_helm", "arxiv")["test"]
prompts = [sample["text"] for sample in ds]

import time
import logging

logger = logging.getLogger(__name__)

def run_experiment_workflow():
    max_retries = 3
    retry_delay = 5  # seconds between retries
    success = False
    
    for attempt in range(max_retries):
        try:
            # Initialize the experiment runner with the configuration and prompts.
            runner = ExperimentRunner(experiment_config, prompts)
            
            # Run the experiment. This call internally uses distributed accelerator,
            # so each process will run it.
            runner.run_torch()
            
            # If not the main process, exit cleanly.
            if not runner.accelerator.is_main_process:
                return
            
            # Only main process continues.
            runner.aggregate_results()
            runner.save_experiment_results()
            
            # Clean up resources.
            runner.teardown()
            
            logger.info(f"Experiment run succeeded on attempt {attempt+1}")
            success = True
            break  # Exit retry loop on success.
        except Exception as e:
            logger.error(f"Experiment run failed on attempt {attempt+1}: {e}", exc_info=True)
            time.sleep(retry_delay)
    
    if not success:
        logger.error("Experiment failed after maximum attempts. Moving on.")
        # Optionally: save a record of the failure so you can re-run later.
    return

# Launch the experiment across the specified number of processes.
notebook_launcher(run_experiment_workflow,
                    num_processes=experiment_config.num_processes)
                    #terminate_on_error=True) # USE THIS IN accelerate.launch.launch()  
              

Launching training on 4 GPUs.
Accelerator set up
Unique experiment id: 0274
TinyLlama/TinyLlama-1.1B-Chat-v1.0 loaded using pytorch, with precision float32
Entering wait barrier: after load_model_tokenizer
wait_for_everyone completed within 10 seconds for after load_model_tokenizer.
Exiting wait barrier: after load_model_tokenizer
Original generate method saved.
Model and tokenizer prepared
Entering wait barrier: after model preparation
wait_for_everyone completed within 10 seconds for after model preparation.

INFO:experiments.experiment_runner:[Process 3889693] Model is on device: cuda:2


INFO:experiments.experiment_runner:[Process 3889692] Model is on device: cuda:1
INFO:experiments.experiment_runner:[Process 3889694] Model is on device: cuda:3


Exiting wait barrier: after model preparation


INFO:experiments.experiment_runner:[Process 3889691] Model is on device: cuda:0


...

INFO:experiments.experiment_runner:[Process 3889694][GPU 3] saved its energy metrics.



Experiment finished
Destroyed process group successfully


INFO:experiments.experiment_runner:Aggregating per-process energy metrics from disk.
INFO:experiments.experiment_runner:Aggregated global energy results successfully from disk.
INFO:experiments.experiment_runner:Saving experiment results
INFO:experiments.experiment_runner:Experiment results saved to results/text_generation_results.json


Starting teardown process...
Process group not initialized.
Emptying CUDA cache...
Running garbage collection...
Teardown process complete.


INFO:__main__:Experiment run succeeded on attempt 1
